In [1]:
from numpy import vstack
import pandas as pd

import random
from random import seed
from random import randint

from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

use this link for dataprep:
    https://scikit-learn.org/stable/modules/preprocessing.html#standardization-or-mean-removal-and-variance-scaling

## 1. import dataset

In [2]:
input_path= "C:/Users/Minaz/OneDrive - HEC Montréal/Travail/Desjardins/R&D/GRAD/Data/German Credit/german.txt"
dataset=pd.read_csv(input_path, sep=" ")
dataset.head()

,At1,At2,At3,At4,At5,At6,At7,At8,At9,At10,...,At12,At13,At14,At15,At16,At17,At18,At19,At20,Y
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [3]:
X_dataset=dataset.drop("Y",axis=1)
y_dataset=dataset.Y

In [4]:
column_list=list(X_dataset.columns)
object_cols =list(X_dataset.select_dtypes(include=['category','object','bool']))
num_cols=[x for x in column_list if x not in object_cols]
print(column_list)
print(object_cols)
print(num_cols)

['At1', 'At2', 'At3', 'At4', 'At5', 'At6', 'At7', 'At8', 'At9', 'At10', 'At11', 'At12', 'At13', 'At14', 'At15', 'At16', 'At17', 'At18', 'At19', 'At20']
['At1', 'At3', 'At4', 'At6', 'At7', 'At9', 'At10', 'At12', 'At14', 'At15', 'At17', 'At19', 'At20']
['At2', 'At5', 'At8', 'At11', 'At13', 'At16', 'At18']


In [5]:
X_dataset.dtypes

At1     object
At2      int64
At3     object
At4     object
At5      int64
At6     object
At7     object
At8      int64
At9     object
At10    object
At11     int64
At12    object
At13     int64
At14    object
At15    object
At16     int64
At17    object
At18     int64
At19    object
At20    object
dtype: object

In [6]:
X_dataset_num=X_dataset.drop(object_cols,axis=1)
X_dataset_cat=X_dataset[object_cols]

## 1. Groupe together the less present modalities

In [7]:
from sklearn.model_selection import train_test_split
class MinaReplace():
    def __init__(self , data_X,data_y, list_of_columns = None , threshold = 0.0002  , valid_size = 0.3):
        self.data_X = data_X             #The X data set
        self.data_y = data_y             #The y data set
        self.threshold = threshold   # The critaria that we use in replace phase
        self.list_of_columns = list_of_columns #The list of the name of  all categroaicl columns in data table. 
        self.valid_size = valid_size  # Represent the proportion of the dataset to include in the valid split.
    
    def find_columns(self): # Find categorical columns; We develop tis function later
        
    
    def train_valid_split(self): # split data in 2 data sets
        
        self.X_train, self.X_valid, self.y_train, self.y_valid = train_test_split(self.data_X,
                                                                                  self.data_y, 
                                                                                  test_size = self.valid_size, 
                                                                                  random_state=42)
    
    def replace_in_train_valid(self , i):
        mask = self.X_train[i].value_counts(normalize=True)< self.threshold
        minority = self.X_train[i].value_counts(normalize=True)[mask].index.tolist() 
        self.d = dict() 
        for val in minority: 
            self.d[val]='Other'  #???? how to add name of column at the end of other? 
        self.X_train[i].replace(self.d,inplace=True) 
        self.X_valid[i].replace(self.d,inplace=True) 


    def main_function(self):
        self.train_valid_split()
        self.find_columns()
        for i in self.list_of_columns:
            self.replace_in_train_valid(i)
        return(self.X_train, self.X_valid, self.y_train, self.y_valid)
            
 

IndentationError: expected an indented block (<ipython-input-7-39a27be91f21>, line 13)

In [8]:
mina0=MinaReplace(data_X=X_dataset,data_y=y_dataset, list_of_columns = object_cols , threshold = 0.05  , valid_size = 0.3)

NameError: name 'MinaReplace' is not defined

In [34]:
X_train_0,X_val_0,y_train_0,y_val_0=mina0.main_function()

C:\Users\Minaz\Anaconda3\lib\site-packages\pandas\core\generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [36]:
X_train_0.At10.value_counts()

A101     630
A103      40
Other     30
Name: At10, dtype: int64

## 2. Onehot encoding
this function retuns a new train and valid dataset with the binary columns for all categorical columns. So this does contain NI the source categorical NOR numeric columns. JUST binary columns for categorical columns

In [9]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
class MinaOneHot():
    def __init__(self , X_train, X_valid, list_of_columns = None ):
        self.X_train = X_train             #The data set
        self.X_valid = X_valid             #The data set
        self.list_of_columns = list_of_columns #The list of the name of  all categroaicl columns in data table. 
    
    def find_columns(self): # Find categorical columns; We develop tis function later
        pass
      
    def onehotencoder(self , i):
        encode_1=LabelEncoder()                     # créer un objet de la classe LabelEncoder
        encode_2=OneHotEncoder(categories='auto')   # créer un objet de la classe OneHotEncoder
        # combiner l'application des deux classes
        self.encode_X_train_array=encode_2.fit_transform(encode_1.fit_transform(self.X_train[i]).reshape(-1,1))
        self.encode_X_valid_array=encode_2.transform(encode_1.transform(self.X_valid[i]).reshape(-1,1))
        
        self.encode_X_train_df=pd.DataFrame(self.encode_X_train_array.toarray(),columns=self.X_train[i].unique())
        self.encode_X_valid_df=pd.DataFrame(self.encode_X_valid_array.toarray(),columns=self.X_valid[i].unique())
        self.encode_X_train_df.drop(columns=self.encode_X_train_df.columns[-1], axis=1, inplace=True)
        self.encode_X_valid_df.drop(columns=self.encode_X_valid_df.columns[-1], axis=1, inplace=True)

    def main_function(self):
        self.find_columns()
        self.X_t=pd.DataFrame()
        self.X_v=pd.DataFrame()
        for i in self.list_of_columns:
            self.onehotencoder(i)
            self.X_t=pd.concat([self.X_t.reset_index(drop=True), self.encode_X_train_df], axis=1)
            self.X_v=pd.concat([self.X_v.reset_index(drop=True), self.encode_X_valid_df], axis=1)
        return(self.X_t,self.X_v)
            

In [10]:
mina1=MinaOneHot(X_train_0,X_val_0, list_of_columns = object_cols)

NameError: name 'X_train_0' is not defined

In [29]:
X_train_1,X_val_1=mina1.main_function()

In [76]:
X_train_1.head()

,A14,A11,A12,A33,A32,A34,A30,A40,A42,A41,...,A122,A143,A141,A153,A152,A173,A174,A172,A192,A201
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [91]:
print(X_train_1.shape)
print(X_val_1.shape)

(700, 37)
(300, 37)


## 3. StandardScaler
this function retuns a train and valid dataset of standardized numerical columns. So contains juste numerical columns

In [93]:
from sklearn.preprocessing import StandardScaler
class MinaStandscaler():
    def __init__(self , X_train, X_valid,list_of_numeric_columns = None):
        self.X_train = X_train             #The data set
        self.X_valid = X_valid             #The data set
        self.list_of_numeric_columns=list_of_numeric_columns
    
    def find_columns(self): # Find categorical columns; We develop tis function later
        self.X_train_num=self.X_train[self.list_of_numeric_columns]
        self.X_valid_num=self.X_valid[self.list_of_numeric_columns]
      
    def stand(self):
        standardscale_1=StandardScaler().fit(self.X_train_num)                    # créer un objet de la classe StandardSclae
        # combiner l'application des deux classes
        standard_X_train_array=standardscale_1.transform(self.X_train_num)
        standard_X_valid_array=standardscale_1.transform(self.X_valid_num)
        
        self.standard_X_train_df=pd.DataFrame(standard_X_train_array,columns=self.list_of_numeric_columns)
        self.standard_X_valid_df=pd.DataFrame(standard_X_valid_array,columns=self.list_of_numeric_columns)

    def main_function(self):
        self.find_columns()
        self.stand()
        X_t_s=self.standard_X_train_df
        X_v_s=self.standard_X_valid_df
        return(X_t_s,X_v_s)
            

In [94]:
mina2=MinaStandscaler(X_train_0,X_val_0,list_of_numeric_columns = num_cols)

In [95]:
X_train_2, X_val_2=mina2.main_function()

In [96]:
print(X_train_2.shape)
print(X_val_2.shape)

(700, 7)
(300, 7)


## Put all together

In [121]:
dic1={2:0}
y_val=pd.DataFrame(y_val_0)
y_val.Y.replace(dic1,inplace=True) 

y_train=pd.DataFrame(y_train_0)
y_train.Y.replace(dic1,inplace=True) 

In [122]:
X_train=pd.concat([X_train_1.reset_index(drop=True), X_train_2], axis=1)
X_train=pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)
X_val=pd.concat([X_val_1.reset_index(drop=True), X_val_2], axis=1)
X_val=pd.concat([X_val.reset_index(drop=True), y_val.reset_index(drop=True)], axis=1)

In [123]:
print(X_train.shape)
print(X_val.shape)

(700, 45)
(300, 45)


# ATTENTION: we need think about
    - the NAs replacing strategy
    - the date columns
    - uper or lower case the values in columns before using 1st function
    

In [127]:
X_train.to_csv("C:/Users/Minaz/OneDrive - HEC Montréal/Travail/Desjardins/R&D/GRAD/Data/German Credit/german_credit_cleaned_train.csv",index=False)

In [126]:
X_val.to_csv("C:/Users/Minaz/OneDrive - HEC Montréal/Travail/Desjardins/R&D/GRAD/Data/German Credit/german_credit_cleaned_val.csv",index=False)